In [2]:
import torch
x = torch.cuda.FloatTensor([3.5])
x.device

device(type='cuda', index=0)

In [3]:
%%timeit

import torch
aa = torch.cuda.FloatTensor([600])
bb = torch.matmul(aa,aa)

33.3 µs ± 860 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

using cuda: Tesla T4
